#### Study initial dataset

In [33]:
import os
from pathlib import Path
from pymatgen.io.cif import CifWriter
from mp_api.client import MPRester
import pandas as pd
import json
import ormsgpack as mp
import numpy as np
from matplotlib_venn import venn2

api_key = os.environ.get("MP_API_KEY")

46744


In [ ]:
# read ids
ids = pd.read_csv("./data/material-data/mp-ids-46744.csv")
ids = [list(ids)[0]] + list(ids.iloc[:, 0])
print(len(ids))

In [11]:
# my venn diagram for two sets
def venn(**kvarg):
    subsets = []
    set_labels = []
    for k, v in kvarg.items():
        subsets.append(set(v))
        set_labels.append(k)
    return venn2(subsets=subsets, set_labels=set_labels)


# get duplicates set of duplicates
def get_duplicates_list(a: list):
    h = dict()
    for x in a:
        h.setdefault(x, 0)
        h[x] += 1
    repeated = list()
    for k, v in h.items():
        if v > 1:
            repeated += [k for i in range(1, v)]
    return repeated


def get_duplicates_set(a: list):
    return set(get_duplicates_list(a))

##### Naive

In [ ]:
with MPRester(api_key) as mpr:
    # naive_docs = mpr.materials.search(material_ids=ids, fields=['material_id'])
    # ValueError: List of material/molecule IDs provided is too long. Consider removing the ID filter to automatically pull data for all IDs and filter locally.
    # fix via chunk loading with const:
    # 10000
    filter_const = 10000
    naive_docs = [
        d
        for k in range(0, len(ids), filter_const)
        for d in mpr.materials.search(
            material_ids=ids[k : k + filter_const], fields=["material_id"]
        )
    ]
    depr_naive_docs = [
        d
        for k in range(0, len(ids), filter_const)
        for d in mpr.materials.search(
            material_ids=ids[k : k + filter_const],
            fields=["material_id"],
            deprecated=True,
        )
    ]

In [ ]:
naive_material_ids = [str(d.material_id) for d in naive_docs]
naive_material_ids_repeated = get_duplicates_set(naive_material_ids)

depr_naive_material_ids = [str(d.material_id) for d in depr_naive_docs]
depr_naive_material_ids_repeated = get_duplicates_set(depr_naive_material_ids)
venn(naive_material_ids=naive_material_ids,
     depr_naive_material_ids=depr_naive_material_ids)

In [ ]:
venn(naive_material_ids=naive_material_ids,
     naive_material_ids_repeated=naive_material_ids_repeated)

In [ ]:
venn(depr_naive_material_ids=depr_naive_material_ids,
     depr_naive_material_ids_repeated=depr_naive_material_ids_repeated)

##### Non-naive

In [ ]:
with MPRester(api_key) as mpr:
    docs = [
        d
        for k in range(0, len(ids), filter_const)
        for d in mpr.materials.search(
            task_ids=ids[k : k + filter_const], fields=["material_id"]
        )
    ]
    depr_docs = [
        d
        for k in range(0, len(ids), filter_const)
        for d in mpr.materials.search(
            task_ids=ids[k : k + filter_const], fields=["material_id"], deprecated=True
        )
    ]

In [ ]:
material_ids = [str(d.material_id) for d in docs]
material_ids_repeated = get_duplicates_set(material_ids)

depr_material_ids = [str(d.material_id) for d in depr_docs]
depr_material_ids_repeated = get_duplicates_set(depr_material_ids)

venn(material_ids=material_ids,
     depr_material_ids=depr_material_ids)

In [ ]:
venn(material_ids=material_ids,
     material_ids_repeated=material_ids_repeated)

In [ ]:
venn(depr_material_ids=depr_material_ids,
     depr_material_ids_repeated=depr_material_ids_repeated)

Knowing the complete set of material_ids it is possible to create a map between task_ids and material_ids

In [ ]:
all_mids = list(set(material_ids).union(set(depr_material_ids)))
mids = list(set(material_ids))
depr_mids = list(set(depr_material_ids))
with MPRester(api_key) as mpr:
    docs_for_t2m = [
        d
        for k in range(0, len(all_mids), filter_const)
        for d in mpr.materials.search(
            material_ids=all_mids[k : k + filter_const],
            fields=["material_id", "task_ids"],
        )
    ]
    depr_docs_for_t2m = [
        d
        for k in range(0, len(all_mids), filter_const)
        for d in mpr.materials.search(
            material_ids=all_mids[k : k + filter_const],
            fields=["material_id", "task_ids"],
            deprecated=True,
        )
    ]

In [ ]:
mids = [str(d.material_id) for d in docs_for_t2m]
depr_mids = [str(d.material_id) for d in depr_docs_for_t2m]
venn(mids=mids, depr_mids=depr_mids)
# No venn intersection => mids are unique

In [ ]:
mtids = [(str(d.material_id), [str(x) for x in d.task_ids]) for d in docs_for_t2m]
depr_mtids = [(str(d.material_id), [str(x) for x in d.task_ids]) for d in depr_docs_for_t2m]

# keep only task_ids from (initial) ids.
sids = set(ids)
redused_mtids = [(m, [x for x in t if x in sids]) for m, t in mtids]
redused_depr_mtids = [(m, [x for x in t if x in sids]) for m, t in depr_mtids]

# flatten
all_tids = []
for m, t in redused_mtids:
    all_tids += t
for m, t in redused_depr_mtids:
    all_tids += t
print(len(get_duplicates_set(all_tids)))

Correspondence between tids and mids strictly obeys the law that there is only one mid for the given tid.

In [ ]:
# compare with (initial) ids

# create map task_id -> material_id
t2m = dict()
for m, t in mtids:
    for x in t:
        t2m[x] = m
for m, t in depr_mtids:
    for x in t:
        t2m[x] = m
found = [x for x in ids if x in t2m]
not_found = [x for x in ids if x not in t2m]
venn(found=found, not_found=not_found)

In [ ]:
# set theoretical image of ids after t2m
img = [t2m[x] for x in ids]

In [ ]:
print(len(set(img)),len(get_duplicates_list(img)),len(get_duplicates_set(img)))
venn(img=img, ids=ids)
# => Actual number of duplicates (data leak) in article implementation could be about 100 ids (~0.2%)
# (~800 and 1.8% in previous version of db)

#### Dataset reimplementation
Creating dataset with data leak, then without. Checking the difference in training results.

##### Load data

In [ ]:
# for individual files
# def save_structure_to_cif(structure, output_file):
#     with open(output_file, "w") as f:
#         cif_writer = CifWriter(structure)
#         cif_writer.write_file(f.name)

In [6]:
# saving utility
basedir = "./data/root/data/"
cifs_file = "./data/root/data/cifs.bin"
props_file = "./data/root/data/props.bin"
t2m_file = "./data/root/data/t2m.bin"

def save_to_bin(data: dict, output_file):
    with open(output_file, "wb") as f:
        f.write(mp.packb(data,option=mp.OPT_SERIALIZE_NUMPY))

In [7]:
# save_to_bin(t2m, t2m_file)

In [35]:
# downloading functionality
fields = [
    "energy_per_atom",
    "formation_energy_per_atom",
    "band_gap",
    "efermi",
    "shear_modulus",  # "k_voigt", "k_reuss", "k_vrh", see https://docs.materialsproject.org/methodology/materials-methodology/elasticity/
    "bulk_modulus",  # "g_voigt","g_reuss", "g_vrh"
    "homogeneous_poisson",
]


def download_by_material_ids(mids):
    with MPRester(api_key) as mpr:
        docs = [
            d
            for k in range(0, len(mids), filter_const)
            for d in mpr.materials.summary.search(
                material_ids=mids[k : k + filter_const],
                fields=fields + ["structure", "material_id"],
            )
        ]
        depr_docs = [
            d
            for k in range(0, len(mids), filter_const)
            for d in mpr.materials.summary.search(
                material_ids=mids[k : k + filter_const],
                fields=fields + ["structure", "material_id"],
                deprecated=True,
            )
        ]
    all_docs = depr_docs + docs
    return all_docs


def save_docs_as_bin(docs, prefix=None):
    props = dict()
    cifs = dict()
    for d in docs:
        mid = str(d.material_id)
        # structures
        cifs[mid] = d.structure.as_dict()

        # properties
        dump = d.model_dump(include=fields)
        # translate shear_modulus and bulk_modulus to k_ and g_ in case they exist
        modulus_types = ["voigt", "reuss", "vrh"]
        if "shear_modulus" in dump and dump["shear_modulus"] is not None:
            for mt in modulus_types:
                dump["k_" + mt] = dump["shear_modulus"][mt]
            dump.pop("shear_modulus")
        if "bulk_modulus" in dump and dump["bulk_modulus"] is not None:
            for mt in modulus_types:
                dump["g_" + mt] = dump["bulk_modulus"][mt]
            dump.pop("bulk_modulus")
        for moduli in ["k_voigt", "k_reuss", "k_vrh", "g_voigt", "g_reuss", "g_vrh"]:
            if moduli in dump:
                dump[moduli] = np.log(
                    dump[moduli]
                )  # training of this property is performed in log scale
        props[mid] = dump
    # print(cifs)
    if prefix is not None:
        save_to_bin(cifs, basedir+prefix+'_cifs.bin')
        save_to_bin(props, basedir+prefix+'_props.bin')
    else:
        save_to_bin(cifs, cifs_file)
        save_to_bin(props, props_file)

In [10]:
# docs = download_by_material_ids(mids=img)
# save_docs_as_bin(docs)

##### Compression

In [11]:
# compression with zstd
import zstandard as zstd

cifs_zipfile = "./data/root/cifs.zstd"
props_zipfile = "./data/root/props.zstd"


def compress_all():
    with open(cifs_file, "rb") as f:
        res = zstd.compress(f.read(), level=10)
    with open(cifs_zipfile, "wb") as f:
        f.write(res)
    with open(props_file, "rb") as f:
        res = zstd.compress(f.read(), level=10)
    with open(props_zipfile, "wb") as f:
        f.write(res)
        
def uncompress_all():
    with open(cifs_zipfile, "rb") as f:
        res = zstd.decompress(f.read())
    with open(cifs_file, "wb") as f:
        f.write(res)
    with open(props_zipfile, "rb") as f:
        res = zstd.decompress(f.read())
    with open(props_file, "wb") as f:
        f.write(res)

In [12]:
# compress_all()
# uncompress_all()

In [13]:
# for individual files
# propfiles = os.listdir(basedir)
# def load_properties_from_json(file):
#     filename = os.path.basename(file)
#     with open(basedir + file, "rb") as f:
#         properties = json.load(f)
#         return (filename, properties)
# propdata = dict()
# for file in propfiles:
#     name,prop=load_properties_from_json(file)
#     for key in prop.keys():
#         propdata.setdefault(key,[])
#         propdata[key].append(prop[key])

##### Training

In [15]:
# print data statistics for current dataset
import ormsgpack as mp

def load_from_bin(file):
    with open(file, "rb") as f:
        properties = mp.unpackb(f.read())
        return properties


full_df = pd.DataFrame(load_from_bin(props_file)).transpose()
full_df.describe()

/home/nodoteve/apps/pythom/miniconda/envs/cgcnn/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/nodoteve/apps/pythom/miniconda/envs/cgcnn/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/nodoteve/apps/pythom/miniconda/envs/cgcnn/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/nodoteve/apps/pythom/miniconda/envs/cgcnn/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,energy_per_atom,formation_energy_per_atom,band_gap,efermi,homogeneous_poisson,k_voigt,k_reuss,k_vrh,g_voigt,g_reuss,g_vrh
count,46055.000000,46055.000000,46632.000000,46632.000000,8802.000000,8470.000000,8272.000000,8391.000000,8740.000000,8748.000000,8731.000000
mean,-9.402272,-1.703857,1.394342,2.901569,0.298029,3.706039,-inf,3.624273,-inf,-inf,-inf
std,7.408867,1.047019,1.659040,2.674768,2.460508,1.320703,NaN,1.344982,NaN,NaN,NaN
min,-83.024212,-4.510049,0.000000,-14.017281,-179.362000,-3.015935,-inf,-2.603690,-inf,-inf,-inf
25%,-8.646992,-2.502314,0.000000,1.084994,0.250000,3.152789,2.921924,3.042139,3.897767,3.852851,3.866691
50%,-7.329141,-1.857242,0.699550,2.640544,0.294000,3.732202,3.596449,3.659064,4.492276,4.464562,4.477780
75%,-5.984656,-0.746991,2.487525,4.612346,0.339000,4.270973,4.214469,4.240405,5.009493,4.993403,5.002230
max,-0.310334,4.310576,16.586400,17.758767,82.881000,32.358331,8.554642,31.665184,25.044718,17.473889,24.351570


In [20]:
# train on all properties for full_df dataset
import numpy as np
import pandas as pd
from subprocess import run

reference_csv = {
    "mp-ids-3402.csv": [
        "k_voigt",
        # "k_reuss",
        # "k_vrh",
        # "g_voigt",
        # "g_reuss",
        # "g_vrh",
        # "homogeneous_poisson",
    ],
    # "mp-ids-27430.csv": ["band_gap"],
    # "mp-ids-46744.csv": ["energy_per_atom", "formation_energy_per_atom", "efermi"],
}


def set_property_to_ids(df: pd.DataFrame, property: str):
    # nids = df[property].isna()
    df[property].dropna().to_csv(
        "./data/root/data/id_prop.csv", index=True, header=False
    )


def prepare_csv(csv: str, df: pd.DataFrame, prop: str):
    set_property_to_ids(df, prop)


t2m = load_from_bin(t2m_file)

def get_df_for_csv(csv:str):
    global t2m, full_df

    ids = pd.read_csv("./data/material-data/" + csv)
    ids = [list(ids)[0]]+list(ids.iloc[:, 0])
    new_df = pd.DataFrame()
    idx = full_df.index
    ms =[t2m[t] for t in ids if t2m[t] in idx]
    new_df = full_df.loc[ms]
    return new_df

In [17]:
df = get_df_for_csv('mp-ids-3402.csv')
df.describe()

/home/nodoteve/apps/pythom/miniconda/envs/cgcnn/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,energy_per_atom,formation_energy_per_atom,band_gap,efermi,homogeneous_poisson,k_voigt,k_reuss,k_vrh,g_voigt,g_reuss,g_vrh
count,3402.000000,3402.000000,3402.000000,3402.000000,2950.000000,2927.000000,2903.000000,2918.000000,2934.000000,2931.000000,2929.000000
mean,-12.147303,-0.952483,0.678811,4.814247,0.282928,3.927095,-inf,3.858064,4.574724,4.493641,4.548808
std,9.521122,0.956267,1.353280,2.744502,0.136696,1.300404,NaN,1.291339,0.985032,0.875161,0.982597
min,-56.285134,-4.223837,0.000000,-4.654093,-3.500000,-0.689155,-inf,-1.309333,1.525839,-0.507498,1.489753
25%,-15.073407,-1.289773,0.000000,3.001713,0.242000,3.374477,3.185918,3.291057,4.054373,4.017881,4.034152
50%,-8.434996,-0.592026,0.000000,5.175226,0.283000,3.978017,3.871034,3.919753,4.705061,4.680305,4.690549
75%,-6.321394,-0.302318,0.742900,6.931447,0.324000,4.461213,4.390522,4.420929,5.144671,5.132065,5.136263
max,-0.858596,2.756673,8.716100,12.401693,4.017000,32.358331,6.520957,31.665184,18.955657,14.689042,18.262511


In [26]:
# train function
env_name = "cgcnn"
def run_process():
    res = run(
        f"conda run -n {env_name} python main.py {basedir}",
        capture_output=True,
        text=True,
        shell=True,
    )
    return res.stdout.split("** MAE ")[-1].replace('\n','')

In [27]:
# train each parameter once
# with deterministic learning
# ! conda env config vars set 
import shutil as st
maedict = dict()
for csv in reference_csv.keys():
    df = get_df_for_csv(csv)

    for prop in reference_csv[csv]:
        prepare_csv(csv, df, prop)
        mae = run_process()
        try:
            st.move("checkpoint.pth.tar", "./trained/" + prop + "_check.pth.tar")
            st.move("model_best.pth.tar", "./trained/" + prop + "_best.pth.tar")
            st.move("test_results.csv", "./trained/" + prop + "_results.csv")
        except:
            pass
        maedict[prop] = mae
maedict

{'k_voigt': '0.388'}

##### Train without data leak

In [28]:
def prepare_csv_no_leak(csv: str, df: pd.DataFrame, prop: str):
    part_df=df[prop].dropna()
    part_df=part_df.groupby(part_df.index).first()
    part_df.to_csv("./data/root/data/id_prop.csv", index=True, header=False)

In [29]:
# train each parameter once
import shutil as st
maedict = dict()
for csv in reference_csv.keys():
    df = get_df_for_csv(csv)

    for prop in reference_csv[csv]:
        prepare_csv_no_leak(csv, df, prop)
        mae = run_process()
        try:
            st.move("checkpoint.pth.tar", "./trained/" + prop + "_check.pth.tar")
            st.move("model_best.pth.tar", "./trained/" + prop + "_best.pth.tar")
            st.move("test_results.csv", "./trained/" + prop + "_results.csv")
        except:
            pass
        maedict[prop] = mae
        # with open("train_outputs.json", "w") as f:
        #     json.dump(resdict, f)
        # pd.DataFrame(maedict, index=['MAE']).transpose().to_csv('train_maes.csv')
maedict

{'k_voigt': '0.407'}

On recent MT db version for all ids:

In [36]:
# (undeprecated cifs and props)
fields = [
    "energy_per_atom",
    "formation_energy_per_atom",
    "band_gap",
    "efermi",
    "k_voigt",
    "k_reuss",
    "k_vrh",
    "g_voigt",
    "g_reuss",
    "g_vrh",
    "homogeneous_poisson",
]
all_props_file = "./data/root/data/db_props.bin"
all_cifs_file = "./data/root/data/db_cifs.bin"
def download_full_mpdb_undeprecated():
    with MPRester(api_key) as mpr:
        docs = [
            d
            for d in mpr.materials.summary.search(
                fields=fields + ["structure", "material_id"],
            )
        ]
    return docs

# download_all()

In [37]:
# all_docs =  download_full_mpdb_undeprecated()
save_docs_as_bin(all_docs,'db')

KeyError: 'shear_modulus'

In [ ]:
all_props_file_zstd = "./data/root/db_props.zstd"
all_cifs_file_zstd = "./data/root/db_cifs.zstd"


def compress_all():
    with open(cifs_file, "rb") as f:
        res = zstd.compress(f.read(), level=10)
    with open(all_props_file_zstd, "wb") as f:
        f.write(res)
    with open(props_file, "rb") as f:
        res = zstd.compress(f.read(), level=10)
    with open(all_props_file_zstd, "wb") as f:
        f.write(res)

compress_all()

def uncompress_all():
    with open(all_cifs_file_zstd, "rb") as f:
        res = zstd.decompress(f.read())
    with open(cifs_file, "wb") as f:
        f.write(res)
    with open(all_props_file_zstd, "rb") as f:
        res = zstd.decompress(f.read())
    with open(props_file, "wb") as f:
        f.write(res)


# uncompress_all()

In [ ]:
# create DF
def load_from_bin(file):
    with open(file, "rb") as f:
        properties = mp.unpackb(f.read())
        return properties
    return None

full_df = pd.DataFrame(load_from_bin(all_props_file)).transpose()
full_df.describe()

In [ ]:
def set_property_to_ids(df: pd.DataFrame, prop: str):
    part_df = df[prop].dropna()
    print(part_df.head())
    part_df = part_df.groupby(part_df.index).first()
    print(part_df.head())
    part_df.to_csv("./data/root/data/id_prop.csv", index=True, header=False)

In [ ]:
# for prop in fields:
#     set_property_to_ids(full_df, prop)
#     res = run(
#         "conda run -n cgcnn2 python main.py --train-ratio 0.6 --val-ratio 0.2 --test-ratio 0.2 ./data/root/data/",
#         capture_output=True,
#         text=True,
#         shell=True,
#     )
#     resdict[prop] = res.stdout
#     maedict[prop] = resdict[prop].split("** MAE ")[-1].replace('\n','')
    
#     print(prop, maedict[prop])

#     st.move("checkpoint.pth.tar", "./trained/" + prop + "_check.pth.tar")
#     st.move("model_best.pth.tar", "./trained/" + prop + "_best.pth.tar")
#     st.move("test_results.csv", "./trained/" + prop + "_results.csv")
        
# with open("train_outputs.json", "w") as f:
#     json.dump(resdict, f)
# pd.DataFrame(maedict).to_csv('train_maes.csv')

In [ ]:
# sort out crystals with warnings:
from pymatgen.core import Structure
from cgcnn.data import get_all_neighbors
ill = set()
with open('./data/root/data/db_cifs.bin','rb') as f:
    cifs = mp.unpackb(f.read())
    for i, row in full_df.iterrows():
        crystal = Structure.from_dict(cifs[i])
        all_nbrs = get_all_neighbors(crystal, r=8, include_index=True)
        all_nbrs = [sorted(nbrs, key=lambda x: x[1]) for nbrs in all_nbrs]
        max_num_nbr=12
        for nbr in all_nbrs:
            if len(nbr) < max_num_nbr:
                ill.add(i)
non_ill_df = full_df.drop(list(ill), axis='index')
set_property_to_ids(non_ill_df, 'formation_energy_per_atom')
non_ill_df.to_csv('non_ill_df.csv', index=True)

In [ ]:
set_property_to_ids(non_ill_df, 'energy_per_atom')

In [ ]:
for prop in fields:
    set_property_to_ids(non_ill_df, prop)
    res = run(
        "conda run -n cgcnn2 python main.py --train-ratio 0.6 --val-ratio 0.2 --test-ratio 0.2 ./data/root/data/",
        capture_output=True,
        text=True,
        shell=True,
    )
    resdict[prop] = res.stdout
    maedict[prop] = resdict[prop].split("** MAE ")[-1].replace('\n','')
    
    print(prop, maedict[prop])

    st.move("checkpoint.pth.tar", "./trained/" + prop + "_check.pth.tar")
    st.move("model_best.pth.tar", "./trained/" + prop + "_best.pth.tar")
    st.move("test_results.csv", "./trained/" + prop + "_results.csv")
        
with open("train_outputs.json", "w") as f:
    json.dump(resdict, f)
pd.DataFrame(maedict).to_csv('train_maes.csv')